# Module 1: Is the hourglass model for gene expression really supported by the data?
### Paper to be examined: 
“A phylogenetically based transcriptome age index mirrors ontogenetic divergence patterns”, Nature 9;468(7325):815-8 (2010)[1]
### Key claim of the paper: 
"Gene expression follows the so-called hourglass pattern observed for morphological features of development, which are most similar to each other in the phylotypic stage in mid-development."

### Schedule:
* H1: General introduction to the paper/motivation
* H2-3: Write code to import the data and start computing transcriptome age index (TAI)
* H4-6: Aim to reproduce figure 1 of the paper – help/scripts will be given if needed.
* H7: Discussion: “Are you convinced of this result? What might have gone wrong?”
* H8: Redo analysis using log-transformed data
* H9: Summarize results (e.g. on this wiki)


### Key bioinformatics concept of this module: 
"Data normalization is important and can impact the results of subsequent analyses!"


# Installation and Setup

* Install the Anaconda distribution of Python 3.x.


# Libraries
Will be using [**GEOparse**](https://geoparse.readthedocs.io/en/latest/usage.html#working-with-geo-accession) for for fetching gene expression data and [**pandas**](https://pandas.pydata.org/pandas-docs/stable/10min.html) for data manipulation and preprocessing.

In [ ]:
########### import necessary packages
import pandas as pd
import numpy as np
import GEOparse
import matplotlib.pyplot as plt

# Read gene expression data 


In [ ]:
############# Download the data
file_name = 'GSE24616'
gse = GEOparse.get_GEO(geo=file_name, destdir="./")

## GSE data structure:
Let's take a look at gene expression data stucture and accession.

**Data Sturcture:**
    - gse.gsms
        - gse.gsms.metadata
        - gse.gsms.name
        - gse.gsms.table
    - gse.gpl
        - gse.gpl.metadata
        - gse.gpl.name
        - gse.gpl.table
        
**GSE file name:** GSE24616

In [ ]:
########### Explore an example of GSE content
print ("GSM example:")
for gsm_name, gsm in gse.gsms.items():
    print ("Name: ", gsm_name)
    print ('*'*100)
    print ("Metadata:"),
    for key, value in gsm.metadata.items():
        print(" - %s : %s" % (key, ", ".join(value)))
    print('*'*100)
    print ("Table data:"),
    print (gsm.table.head())
    break

In [ ]:
print ("GPL example:")
for gpl_name, gpl in gse.gpls.items():
    print ("Name: ", gpl_name)
    print ('*'*100)
    print ("Metadata:"),
    for key, value in gpl.metadata.items():
        print (" - %s : %s" % (key, ", ".join(value)))
    print ('*'*100)
    print ("Table data:"),
    print (gpl.table.head())
    break


# Read age index data file

# Pre processing gene expression data:
Gene expression data needs to be extracted from GSE data structure. Preprocessing steps are:
1. Extract the metadata
2. Extract the gene expression data
3. Add age index data to the gene expression data
4. Get average for the genes with multiple probesets
5. Select mixed and female samples 
6. Get the average gene expression for similar time points¶

## 1) Extract  metadata of samples in gene expression data
Complementary information about the samples is stores in  **gsm.metadata** including sex, developmental stage and the sample name. A sample metadata looks like:

"characteristics_ch1 : strain: wild type, developmental stage: adult, developmental timing: 1y9m, gender: mixed, number of individuals per sample: 2"

All these infomations are stored in an string and we need to extract them by some String Formatting Operations.

In [ ]:
############### Extract GSE metadata
characteristics = {"stage":[],"time":[],"sex":[],"sample_name":[]}
for gsm_name, gsm in sorted(gse.gsms.items()): # gsm: metadata, columns, table
    characteristics["stage"].append(gsm.metadata['characteristics_ch1'][1].split(":")[1].strip())
    characteristics["time"].append(gsm.metadata['characteristics_ch1'][2].split(":")[1].strip())
    characteristics["sex"].append(gsm.metadata['characteristics_ch1'][3].split(":")[1].strip())
    characteristics["sample_name"].append(gsm_name)
char_df = pd.DataFrame(characteristics,index = characteristics["sample_name"])
char_df.head()

## 2) Extract the gene expression data

In [ ]:
############### Extract the gene expression data
data = gse.pivot_samples('VALUE') 
############# Add ProbeID as the index of gene expression dataframe
gpl = list(gse.gpls.values())[0]
data.set_index(gpl.table.SPOT_ID,inplace=True)
############ Let's look at the gene expression data
data.head()
plt.imshow(np.log(data.values),aspect='auto')
plt.show()

## 3) Add age index data to the gene expression data

## 4) Get average for the genes with multiple probesets

## 5) Select mixed and female samples

## 6) Get the average gene expression for similar time points

# TAI Calculation

# Save the pre-processed expression data to a file